## Data Akuisisi

In [ ]:
# !pip uninstall -y kagglehub
# !pip install kagglehub==0.3.13

In [2]:
import kagglehub
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pastikan file kaggle.json ada di C:\Users\<namaakun>\.kaggle

In [3]:
api = KaggleApi()
api.authenticate()

dataset = "uom190346a/sleep-health-and-lifestyle-dataset"
folder_tujuan = "data"
api.dataset_download_files(dataset, path=folder_tujuan, unzip=True)

print(f"Dataset berhasil di-download ke folder: {folder_tujuan}")

Dataset URL: https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset
Dataset berhasil di-download ke folder: data


In [4]:
df = pd.read_csv(f'data/Sleep_health_and_lifestyle_dataset.csv')
df.head(2)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


Sleep Disorder: The presence or absence of a sleep disorder in the person (None, Insomnia, Sleep Apnea).

| Kolom                   | Deskripsi                                                           | Skala   |
| ----------------------- | ------------------------------------------------------------------- | ------- |
| Gender                  | Jenis kelamin orang (Laki-laki/Perempuan)                           | Nominal |
| Age                     | Usia orang dalam tahun                                              | Rasio   |
| Occupation              | Pekerjaan atau profesi orang                                        | Nominal |
| Sleep Duration          | Jumlah jam tidur per hari                                           | Rasio   |
| Quality of Sleep        | Penilaian subjektif kualitas tidur (1-10)                           | Ordinal |
| Physical Activity Level | Jumlah menit aktivitas fisik per hari                               | Rasio   |
| Stress Level            | Penilaian subjektif tingkat stres (1-10)                            | Ordinal |
| BMI Category            | Kategori BMI (Kurus, Normal, Kelebihan Berat, dll.)                 | Nominal |
| Blood Pressure          | Tekanan darah (sistolik/diastolik, mmHg)                            | Rasio   |
| Heart Rate              | Detak jantung istirahat per menit                                   | Rasio   |
| Daily Steps             | Jumlah langkah per hari                                             | Rasio   |
| Sleep Disorder          | Ada atau tidaknya gangguan tidur (Tidak ada, Insomnia, Sleep Apnea) | Nominal |
| Systolic Pressure       | Tekanan darah sistolik                                              | Rasio   |
| Diastolic Pressure      | Tekanan darah diastolik                                             | Rasio   |


In [6]:
df['Sleep Disorder'] = df['Sleep Disorder'].replace({'None': 'No Disorder'})
df[['Systolic Pressure', 'Diastolic Pressure']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)
df_corr = df.drop(columns=['Person ID', 'Blood Pressure'])

df_corr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   374 non-null    object 
 1   Age                      374 non-null    int64  
 2   Occupation               374 non-null    object 
 3   Sleep Duration           374 non-null    float64
 4   Quality of Sleep         374 non-null    int64  
 5   Physical Activity Level  374 non-null    int64  
 6   Stress Level             374 non-null    int64  
 7   BMI Category             374 non-null    object 
 8   Heart Rate               374 non-null    int64  
 9   Daily Steps              374 non-null    int64  
 10  Sleep Disorder           155 non-null    object 
 11  Systolic Pressure        374 non-null    int64  
 12  Diastolic Pressure       374 non-null    int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 38.1+ KB


### Analisis Korelasi

otw